# Dimensionality Reduction and Random Forests for Breed_category_prediction

## Importing the libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
test_path = '/content/drive/My Drive/Datasets/Hacker_Earth_Challenges/Adopt_a_pet/Dataset/test.csv'
train_path = '/content/drive/My Drive/Datasets/Hacker_Earth_Challenges/Adopt_a_pet/Dataset/train.csv'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [ ]:
df = pd.read_csv(train_path)
from sklearn.impute import SimpleImputer
imp=SimpleImputer(missing_values=np.nan, strategy="mean" )
df["condition"]=imp.fit_transform(df[["condition"]]).ravel()
df = df.drop(['color_type'], axis=1)
print(df.info())
X = df.iloc[:, 3:9].values
y = df.iloc[:, 8:9].values

In [ ]:
# Faster Independent Component analysis
from sklearn.decomposition import FastICA
transformer = FastICA(n_components=2,random_state=0)
X_transformed = transformer.fit_transform(X,y)
X_transformed.shape

(18834, 2)

## LDA less accuracy 88%

In [ ]:
# #Linear Discriminant analysis
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y[:,0], test_size = 0.2, random_state = 0)
# lda = LDA(n_components = 2)
# X_train = lda.fit_transform(X_train, y_train)
# X_test = lda.transform(X_test)

In [ ]:
print(X[0],y[0])
print(X_transformed[0],y[0])

## Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size = 0.2, random_state = 0)

## Training Randomforests on the Training set

In [ ]:
from sklearn.ensemble import RandomForestClassifier
r_classifier = RandomForestClassifier(n_estimators=200, criterion='entropy', random_state=0) #94% accuracy 300trees FastICA
r_classifier.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
Y = df.iloc[:,9:11].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
m_classifier = OneVsRestClassifier(classifier)
clf = MultiOutputClassifier(classifier, n_jobs=-1)
clf.fit(X_train, y_train)
clf.predict(X[-2:])

In [ ]:
# compare standalone models for binary classification
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier

from matplotlib import pyplot



# get a list of models to evaluate
def get_models():
	models = dict()
	models['rf'] = RandomForestClassifier(n_estimators=200)
	models['cart'] = DecisionTreeClassifier()
	return models

# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)
	scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
	return scores


# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X_train, y_train)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

## Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
model = r_classifier
y_pred = model.predict(X_test)
#for 1st target value
cm = confusion_matrix(y_test, y_pred)
print(cm)
score = accuracy_score(y_test, y_pred)*100
print(score)
#for 2nd target value
# cm2 = confusion_matrix(y_test[:,1], y_pred[:,1])
# print(cm2)
# score2 = accuracy_score(y_test[:,1], y_pred[:,1])*100
# print(score2)

[[1698   25   54]
 [  13 1658    5]
 [  95    7  212]]
94.71728165649057


In [ ]:
y_pred.shape

In [ ]:
from sklearn.metrics import f1_score
score = f1_score(y_test, y_pred, average='weighted') * 100
# score2 = f1_score(y_test[:,1], y_pred[:,1], average='weighted') * 100
print(score)
    # , score2)

94.58305428831136


## Applying k-Fold Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = r_classifier, X = X_train, y = y_train[:,0], cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
accuracies = cross_val_score(estimator = r_classifier, X = X_train, y = y_train[:,1], cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))

# Grid Search

Best Accuracy: 89.02 %
Best Parameters: {'n_estimators': 90}

Best Accuracy: 89.04 %
Best Parameters: {'n_estimators': 300}

Best Accuracy: 95.11 %
Best Parameters: {'n_estimators': 200}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = {'n_estimators': [180,200,230,250,270,350]
}
grid_search = GridSearchCV(estimator = r_classifier,
                           param_grid = grid,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

#Preprocess the test set for submission

In [ ]:
test_Set = pd.read_csv(test_path)
print(test_Set.info())
from sklearn.impute import SimpleImputer
imp=SimpleImputer(missing_values=np.nan, strategy="mean" )
test_Set["condition"]=imp.fit_transform(test_Set[["condition"]]).ravel()
test_Set = test_Set.drop(['color_type'], axis=1)
X_test_final = test_Set.iloc[:, 3:9].values
X_test_final = transformer.fit_transform(X_test_final)

In [ ]:
X_test_final[0]

array([ 0.01319199, -0.00647494])

In [ ]:
y_pred_breedcategory = r_classifier.predict(X_test_final)
# y_pred_breedcategory
predicted_breed_category = [int(i) for i in y_pred_breedcategory]

In [ ]:
print(set(predicted_breed_category))

{0, 1, 2}


In [ ]:
d = {'pet_id':test_Set['pet_id'], 'breed_category':predicted_breed_category}

In [ ]:
datafra = pd.DataFrame(d)
# d.to_csv('Submission.csv', index=False)
datafra

,pet_id,breed_category
0,ANSL_75005,0
1,ANSL_76663,1
2,ANSL_58259,1
3,ANSL_67171,0
4,ANSL_72871,1
...,...,...
8067,ANSL_66809,1
8068,ANSL_59041,2
8069,ANSL_60034,1
8070,ANSL_58066,1


In [ ]:
datafra.to_csv('breed_category.csv',index=False) # got 80 marks